# Capstone Project
#### By: Ajay Sharma

## Introduction
This project is to refresh the concepts I have learned throughout this course. I will also demonstrate how I can use the data analysis skills to solve a real world problem through this capstone project 

In [1]:
import pandas as pd
import numpy as np
#!conda install -c conda-forge geopy --yes

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge pgeocode --yes
import pgeocode
import requests
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Download Toronto Neighborhood data from Wikipedia

In [2]:
#! conda install beautifulsoup4
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re


def parse_data():
	page = urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
	html = page.read().decode("utf-8")
	soup = BeautifulSoup(html, 'html.parser')
	table_data = soup.find("table").findAll("td")
	parsed_data = []
	for data in table_data:
		new_data = []
		postal_code = data.find("b").text
		borough_data = data.find("span")
		if postal_code and borough_data.text != "Not assigned":
			borough_data_vals = re.split('[(]',borough_data.text)
			borough = borough_data_vals[0]
			neighborhoods = borough_data_vals[1].replace(" / ", ",").replace(")", "")
			new_data.append(postal_code)
			new_data.append(borough)
			new_data.append(neighborhoods)
			parsed_data.append(new_data)
	return parsed_data

In [3]:
data = parse_data()

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park,Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor,Lawrence Heights'],
 ['M7A', "Queen's Park", 'Ontario Provincial Government'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Malvern,Rouge'],
 ['M3B', 'North York', 'Don MillsNorth'],
 ['M4B', 'East York', 'Parkview Hill,Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B',
  'Etobicoke',
  'West Deane Park,Princess Gardens,Martin Grove,Islington,Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill,Port Union,Highland Creek'],
 ['M3C', 'North York', 'Don MillsSouth'],
 ['M4C', 'East York', 'Woodbine Heights'],
 ['M5C', 'Downtown Toronto', 'St. James Town'],
 ['M6C', 'York', 'Humewood-Cedarvale'],
 ['M9C',
  'Etobicoke',
  'Eringate,Bloordale Gardens,Old Burnhamthorpe,Markland Wood'],
 ['M1E', 'Scarborough', 'Guildwood,Morni

## 1. EXPLORE DATA

In [4]:
neighborhoods = pd.DataFrame(data=data)
headers = ["PostalCode", "Borough", "Neighborhood"]
neighborhoods.columns = headers
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### Prepare data for visualization

In [5]:
def get_lat(row):
    postal_code=row['PostalCode']
    nomi = pgeocode.Nominatim("ca")
    location = nomi.query_postal_code(postal_code)
    latitude = location.latitude
    return latitude

def get_lon(row):
    postal_code=row['PostalCode']
    nomi = pgeocode.Nominatim("ca")
    location = nomi.query_postal_code(postal_code)
    longitude = location.longitude
    return longitude

In [6]:
CLIENT_ID = 'S04PZERPPBIQX2VEEOIBK3YYJGLOQTG5KNPOPAUKOVW0OHBZ' # your Foursquare ID
CLIENT_SECRET = '33UQDTS2GD24HTWQA4KXGY0KCDYJJBRTEXVHDJXFFJHGCE52' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [7]:
neighborhoods["latitude"]= neighborhoods.apply(get_lat, axis = 1)
neighborhoods["longitude"]= neighborhoods.apply(get_lon, axis = 1)

In [8]:
neighborhoods['latitude'] = neighborhoods['latitude'].replace(np.nan, 43.754500)
neighborhoods['longitude'] = neighborhoods['longitude'].replace(np.nan, -79.330000)

In [9]:
# create map of Toronto using latitude and longitude values
latitude = 43.7182197
longitude = -79.4482688
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Add Neighborhood venue data

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        results = results["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['latitude'],longitudes=neighborhoods['longitude'])

Parkwoods
Victoria Village
Regent Park,Harbourfront
Lawrence Manor,Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern,Rouge
Don MillsNorth
Parkview Hill,Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park,Princess Gardens,Martin Grove,Islington,Cloverdale
Rouge Hill,Port Union,Highland Creek
Don MillsSouth
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate,Bloordale Gardens,Old Burnhamthorpe,Markland Wood
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Wilson Heights,Downsview North
Thorncliffe Park
Richmond,Adelaide,King
Dufferin,Dovercourt Village
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
The Danforth  East
Harbourfront East,Union Station,Toronto Islands
Little Portugal,Trinity
Kennedy Park,Ionview,East Birchmount Park
Bayview Village
DownsviewEast  
The Danforth West,Riverdale
Toronto Domin

In [15]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.7545,-79.3300,Brookbanks Pool,43.751389,-79.332184,Pool
3,Parkwoods,43.7545,-79.3300,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
...,...,...,...,...,...,...,...
2148,"Mimico NW,The Queensway West,South of Bloor,Ki...",43.6256,-79.5231,Burrito Boyz,43.626657,-79.526349,Burrito Place
2149,"Mimico NW,The Queensway West,South of Bloor,Ki...",43.6256,-79.5231,Kingsway Boxing Club,43.627254,-79.526684,Gym
2150,"Mimico NW,The Queensway West,South of Bloor,Ki...",43.6256,-79.5231,Solmaz,43.626773,-79.527259,Food & Drink Shop
2151,"Mimico NW,The Queensway West,South of Bloor,Ki...",43.6256,-79.5231,Buon Giorno Cafe,43.622801,-79.519322,Italian Restaurant


In [16]:

toronto_venues.head()

(2153, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.7545,-79.3300,Brookbanks Pool,43.751389,-79.332184,Pool
3,Parkwoods,43.7545,-79.3300,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


## 2. SEGMENT DATA
### Group data by Neighborhood

In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood,Long Branch",7,7,7,7,7,7
"Bathurst Manor,Wilson Heights,Downsview North",6,6,6,6,6,6
Bayview Village,5,5,5,5,5,5
"Bedford Park,Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
WillowdaleWest,3,3,3,3,3,3
Woburn,1,1,1,1,1,1
Woodbine Heights,5,5,5,5,5,5


#### Let's find out how many unique categories can be curated from all the returned venues

In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 259 uniques categories.


### Explore data for each Neighborhood

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [20]:
toronto_onehot.shape

(2153, 259)

Next, group rows by neighborhood and by mean of the frequency of occurrence of each category¶

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor,Wilson Heights,Downsview North",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park,Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,WillowdaleWest,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,York Mills West,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Look at the new size¶

In [22]:
toronto_grouped.shape

(101, 259)

Let's print each neighborhood along with the top 5 most common venues¶

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt ----
                       venue  freq
0  Latin American Restaurant  0.25
1            Badminton Court  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4              Movie Theater  0.00


----Alderwood,Long Branch----
               venue  freq
0        Coffee Shop  0.14
1        Pizza Place  0.14
2  Convenience Store  0.14
3                Pub  0.14
4           Pharmacy  0.14


----Bathurst Manor,Wilson Heights,Downsview North----
                       venue  freq
0  Middle Eastern Restaurant  0.17
1                Pizza Place  0.17
2        Fried Chicken Joint  0.17
3              Deli / Bodega  0.17
4   Mediterranean Restaurant  0.17


----Bayview Village----
         venue  freq
0  Gas Station   0.2
1  Flower Shop   0.2
2        Trail   0.2
3         Park   0.2
4      Dog Run   0.2


----Bedford Park,Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1         Pizza Place  0.08
2      Sandwich Place  0.08
3   

4      Chinese Restaurant  0.09


----Studio District----
            venue  freq
0     Coffee Shop  0.14
1           Diner  0.14
2             Gym  0.14
3            Park  0.14
4  Baseball Field  0.14


----Summerhill West,Rathnelly,South Hill,Forest Hill SE,Deer Park----
                venue  freq
0         Coffee Shop  0.33
1  Light Rail Station  0.33
2         Supermarket  0.17
3        Liquor Store  0.17
4         Yoga Studio  0.00


----The Annex,North Midtown,Yorkville----
            venue  freq
0     Coffee Shop  0.09
1  Sandwich Place  0.09
2            Café  0.09
3     Flower Shop  0.04
4    Liquor Store  0.04


----The Beaches----
               venue  freq
0  Health Food Store  0.14
1             Bakery  0.14
2                Pub  0.14
3              Trail  0.14
4   Asian Restaurant  0.14


----The Danforth  East----
               venue  freq
0   Greek Restaurant  0.25
1  Convenience Store  0.25
2       Intersection  0.25
3               Park  0.25
4      Movie Theater  

First, let's write a function to sort the venues in descending order.

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's put that into a pandas dataframe
Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [25]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Badminton Court,Breakfast Spot,Latin American Restaurant,Skating Rink,Farmers Market,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Women's Store
1,"Alderwood,Long Branch",Pizza Place,Gym,Sandwich Place,Coffee Shop,Pub,Pharmacy,Convenience Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
2,"Bathurst Manor,Wilson Heights,Downsview North",Pizza Place,Mediterranean Restaurant,Middle Eastern Restaurant,Coffee Shop,Deli / Bodega,Fried Chicken Joint,Falafel Restaurant,Escape Room,Ethiopian Restaurant,Event Space
3,Bayview Village,Flower Shop,Dog Run,Gas Station,Trail,Park,Women's Store,Event Space,Escape Room,Ethiopian Restaurant,Falafel Restaurant
4,"Bedford Park,Lawrence Manor East",Italian Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Butcher,Café,Liquor Store,Restaurant,Sushi Restaurant,Fast Food Restaurant


## 3. DATA CLUSTERING

Run k-means to cluster the neighborhood into 5 clusters.

In [26]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 3, 3, 3, 3, 3, 3, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [42]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,2.0,Food & Drink Shop,Pool,Park,Construction & Landscaping,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Donut Shop
1,M4A,North York,Victoria Village,43.7276,-79.3148,3.0,Pizza Place,Portuguese Restaurant,Coffee Shop,Intersection,Park,Hockey Arena,Field,Filipino Restaurant,Fast Food Restaurant,Fish & Chips Shop
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.6555,-79.3626,3.0,Coffee Shop,Breakfast Spot,Thrift / Vintage Store,Food Truck,Spa,Event Space,Beer Store,Electronics Store,Restaurant,Bakery
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.7223,-79.4504,3.0,Clothing Store,Coffee Shop,Restaurant,Women's Store,Sandwich Place,Toy / Game Store,Jewelry Store,Juice Bar,Bakery,Food Court
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889,3.0,Sushi Restaurant,Italian Restaurant,Creperie,Theater,Burrito Place,Martial Arts School,Bubble Tea Shop,Park,Café,Mexican Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.6518,-79.5076,3.0,Bank,Breakfast Spot,Sushi Restaurant,Bakery,Pub,Liquor Store,Bar,Restaurant,Coffee Shop,Burger Joint
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830,3.0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Yoga Studio,Hotel,Men's Store,Mediterranean Restaurant,Fast Food Restaurant,Dance Studio
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.7804,-79.2505,3.0,Restaurant,Coffee Shop,Yoga Studio,Italian Restaurant,Martial Arts School,Breakfast Spot,Bookstore,Sushi Restaurant,Japanese Restaurant,Bank
101,M8Y,Etobicoke,"Old Mill South,King's Mill Park,Sunnylea,Humbe...",43.6325,-79.4939,2.0,Park,Baseball Field,Farmers Market,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Women's Store,Dumpling Restaurant


In [ ]:
#drop any rows with nan values
toronto_merged[toronto_merged["Cluster Labels"].isnull()]
toronto_merged = toronto_merged.dropna()
  
# To reset the indices 
toronto_merged = toronto_merged.reset_index(drop = True)

### Visualize the Clusters

In [55]:
  
# Convert float values to int
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)
toronto_merged.head()
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters